In [14]:
%load_ext autoreload
%autoreload 2

import sys; sys.path.insert(0, '../../..') # add parent folder path where lib folder is

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:

import os

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from ml.models import common
from utils import helper, config, rayer, kaggle_dataset_helper
from ml.xai.non_model import simulate_knockoffs, KnockoffSetting


In [3]:
# rayer.get_global_cluster()
# rayer.get_local_cluster()

In [10]:
def use_tokamat_ds():
    df = helper.get_tokamat_dataset()
    df = df.reset_index()

    df = common.label_encode(df)
    df = df.fillna(-1)

    potential_targets = ['WTOT', 'WTH', 'PLTH']
    df_y = df[potential_targets[0]]

    df_X = df[df.columns[~df.columns.isin(potential_targets)]]
    df_X = df_X.drop(['TOK_ID', 'LCUPDATE', 'DATE', 'NEL', 'ENBI'], axis = 1)

    
    return df_X, df_y
    # return train_test_split(X, y, test_size=0.33)


def use_covid_ds():
    df_X, df_y = helper.get_covid_dataset()
    df_X = df_X.drop(['location'], axis = 1)

    return df_X, df_y
    # return train_test_split(df_X, df_y, test_size=0.33)


def use_transaction_predictions_ds():
    ds_train, ds_test = kaggle_dataset_helper.get_transaction_predictions_dataset()
    ds_train = common.label_encode(ds_train)
    ds_test = common.label_encode(ds_test)

    ds_train = ds_train.fillna(-1)
    ds_test = ds_test.fillna(-1)

    df_X = ds_train.loc[:, ds_train.columns != 'target']
    df_y = ds_train['target']

    return df_X, df_y
    



def use_house_pricing_ds():
    ds_train, ds_test = kaggle_dataset_helper.get_house_prices_dataset()
    ds_train = common.label_encode(ds_train)
    ds_test = common.label_encode(ds_test)

    ds_train = ds_train.fillna(-1)
    ds_test = ds_test.fillna(-1)


    df_X = ds_train.loc[:, ds_train.columns != 'SalePrice']
    df_y = ds_train['SalePrice']

    return df_X, df_y

    


In [11]:
df_X, df_y = use_covid_ds()

In [16]:
np.random.seed(KnockoffSetting.SEED)

X = df_X
y = df_y


# param
# simmulate(param, 0, X, y)

In [17]:
df = simulate_knockoffs( itr=2, df_X=X, df_y=y, fdr=0.1)

itr: 1, ksampler:gaussian, fstat:lasso
itr: 2, ksampler:gaussian, fstat:lasso
itr: 1, ksampler:gaussian, fstat:ridge
itr: 2, ksampler:gaussian, fstat:ridge
itr: 1, ksampler:gaussian, fstat:randomforest
itr: 2, ksampler:gaussian, fstat:randomforest


NameError: name 'results' is not defined

In [16]:
df.sort_values(['W'], ascending=False)

,rejections,W,attr
0,0.0,620711.473263,new_cases_per_million
30,1.0,174014.116558,Cardiovascular diseases
20,1.0,31324.059838,Alcohol use disorders
35,0.0,20173.867311,Acute hepatitis
12,1.0,14064.967651,Interpersonal violence
39,1.0,11994.707461,air_polution
36,0.0,7893.854539,Alzheimer's disease and other dementias
11,0.0,6956.630812,Drowning
29,0.0,6252.844541,Protein-energy malnutrition
34,1.0,6087.138993,Digestive diseases


In [59]:
df_results = pd.DataFrame()        

for result in results:
    df_results = pd.concat([df_results, pd.DataFrame(result)], axis=0)
    
df_results.columns = ['rej', 'w', 'itr']

In [60]:
df_results

,rej,w,itr
0,1.0,0.004988,1.0
1,0.0,0.000001,1.0
2,1.0,9.804645,1.0
3,0.0,0.000001,1.0
4,1.0,0.000080,1.0
...,...,...,...
35,0.0,-59238.064386,2.0
36,0.0,18057.707824,2.0
37,0.0,-1300.593139,2.0
38,0.0,60.379812,2.0


In [54]:
pd.concat([pd.DataFrame(results[0]), pd.DataFrame(results[1])])

,0,1,2
0,1.0,0.004988,1.0
1,0.0,0.000001,1.0
2,1.0,9.804645,1.0
3,0.0,0.000001,1.0
4,1.0,0.000080,1.0
...,...,...,...
35,0.0,-0.000002,2.0
36,0.0,-0.000003,2.0
37,0.0,-0.000001,2.0
38,0.0,-0.000004,2.0


In [ ]:
for base_model in self.base_models:
    lazy_results.append(__run_discoveries__.remote(base_model, score_func, threshold))


self.base_models = ray.get(lazy_results)


In [25]:

# for ksampler in knockoffsettings.KSAMPLER:
ksampler = knockoffsettings.KSAMPLER[0]
fstat = knockoffsettings.FSTATS[0]
fdr = knockoffsettings.FDR


In [26]:
kfilter = KnockoffFilter(
    ksampler = ksampler,
    fstat = fstat,
)

rejections = kfilter.forward(
    X = X,
    y = y,
    Sigma = Sigma,
    fdr = fdr
)

In [27]:
rejections

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)

In [ ]:


# print( 'itr: ' + str(index) + ', ksampler:' + str(ksampler) + ', fstat:' + str(fstat) )

if ksampler != 'metro':
    kfilter = KnockoffFilter(
        ksampler = ksampler,
        fstat = fstat,
    )

    rejections = kfilter.forward(
        X = X,
        y = y,
        Sigma = Sigma,
        fdr = fdr
    )